# 第3回 (11/16) Chainerを使ってDeep Learning手法を実装しよう

# Part 1: 前回のMLPをChainerで書いてみよう
- Chainer:  日本のPreffered Networksという企業が開発したDeep Learning用ライブラリ。


- 前回のように、順伝播、誤差逆伝播、パラメータ更新のそれぞれをプログラムする必要はない。
- 必要なのは、ネットワーク構造の定義と、順伝播(=入力x, tからlossを計算する過程)の定義のみ。
- 面倒なことはchainer.optimizerというやつがたったの1行でやってくれる。

In [34]:
import numpy as np
import chainer
from chainer import cuda, Variable, Chain, optimizers
import chainer.functions as F
import chainer.links as L

## MNISTデータの読み込み
- 前回と違ってラベルはone-of-k表現にしなくてOK。

In [32]:
# Load MNIST data
dir = '/data/ishimochi0/dataset/mnist/'

X_train = np.loadtxt(dir + 'train-images.txt').astype(np.float32).reshape((-1, 784)) / 255.
y_train = np.loadtxt(dir + 'train-labels.txt').astype(np.int32)
N_train = len(X_train)

X_test= np.loadtxt(dir + 'test-images.txt').astype(np.float32).reshape((-1, 784)) / 255.
y_test = np.loadtxt(dir + 'test-labels.txt').astype(np.int32)
N_test = len(X_test)

## MLPクラス
- init関数 = ネットワーク構造の定義
 - 線形層はL.Linear(in_dim, out_dim)で定義する。
 - 活性化関数のことはここでは記述しない。更新するパラメータを持ったものだけを書く。
 
 
- call関数 = 順伝播の定義
 - 活性化関数はF.sigmoid, F.reluのように呼び出す。
 - F.softmax_cross_entropyがsoftmaxとlossの計算をセットでやってくれるので、self.yはsoftmaxを掛ける前の値であることに注意。

In [39]:
class MLP(Chain):
    def __init__(self):
        super(MLP, self).__init__()
        with self.init_scope():
            self.fc1 = L.Linear(784, 1000)
            self.fc2 = L.Linear(1000, 1000)
            self.fc3 = L.Linear(1000, 10)
    
    def __call__(self, x, t):
        h = F.sigmoid(self.fc1(x))
        h = F.sigmoid(self.fc2(h))
        self.y = self.fc3(h)
        self.loss = F.softmax_cross_entropy(self.y, t)
        return self.loss

## モデルの定義とoptimizerの設定
- 前回実装した最適化手法はパラメータを勾配方向に更新するだけのシンプルな手法で、確率的勾配降下法(Stochastic Gradient Decent: SGD)と呼ばれる。
- 前回の実装ではlr=0.01としたが、これは今回の実装ではlr=1.0に相当する(batchsize=100の場合)。前回の実装ではミニバッチ内の全データの勾配の「合計」を用いてパラメータを更新していたが、Chainerでは「平均」を用いるので、同じlrではパラメータの更新量が小さくなる。というか本来はそうするべき？

In [40]:
model = MLP() # モデル作成
model.to_gpu() # GPUに載せる
optimizer = optimizers.SGD(lr=1.0)
optimizer.setup(model)

## MNISTの学習
- 前回とほとんど同じです！ただし注意点。
 - chainerでは、入力変数は"Variable型"というやつに変形しておく必要がある。
 - そして、入力に対して順伝播が計算されていくが、その過程で生じる変数も全部Variable型。出力yや、lossもVariable型。
 - Variable型のデータから中身(=numpy array)を取り出すには、".data"とする。

例:

In [41]:
x_data = np.array([5], dtype=np.float32)
x = Variable(x_data)

print(x) # これだと意味不明なアドレスが表示される
print(x.data) # 中身の取得

variable([ 5.])
[ 5.]


In [43]:
n_epoch = 20
batchsize = 100

for epoch in range(n_epoch):
    print('epoch', epoch)
    
    # Training
    sum_loss = 0
    pred_y = []
    perm = np.random.permutation(N_train)
    
    for i in range(0, N_train, batchsize):
        # ミニバッチの作成。GPUに送った後、Variable型に変える
        x = Variable(cuda.to_gpu(X_train[perm[i: i+batchsize]]))
        t = Variable(cuda.to_gpu(y_train[perm[i: i+batchsize]]))
        
        optimizer.update(model, x, t) # 順伝播、誤差逆伝播、パラメータの更新
        sum_loss += cuda.to_cpu(model.loss.data) * len(x.data) # Variable型の中身を取得して、CPUに戻す (戻さなくても大丈夫なこともある)
        pred_y.extend(np.argmax(cuda.to_cpu(model.y.data), axis=1)) # yは前回と違ってsoftmax掛ける前だけど問題ない
        # ミニバッチ毎の識別率はF.accuracy(y, t)でも計算できるけどカウンタの設置が必要だし応用が利かないので却下
        
    loss = sum_loss / N_train
    accuracy = np.sum(np.eye(10)[pred_y] * np.eye(10)[y_train[perm]]) / N_train
    print('Train loss', loss, '| accuracy', accuracy)  
    
    
    # Testing
    sum_loss = 0
    pred_y = []
    
    for i in range(0, N_test, batchsize):
        x = Variable(cuda.to_gpu(X_test[i: i+batchsize]))
        t = Variable(cuda.to_gpu(y_test[i: i+batchsize]))
        
        sum_loss += cuda.to_cpu(model(x, t).data) * len(x.data) # 順伝播
        pred_y.extend(np.argmax(cuda.to_cpu(model.y.data), axis=1))

    loss = sum_loss / N_test
    accuracy = np.sum(np.eye(10)[pred_y] * np.eye(10)[y_test]) / N_test
    print('Test loss', loss, '| accuracy', accuracy)

epoch 0
Train loss 0.0377430974972 | accuracy 0.988666666667
Test loss 0.0736006769713 | accuracy 0.9761
epoch 1
Train loss 0.034131549179 | accuracy 0.98965
Test loss 0.066456923708 | accuracy 0.9785
epoch 2
Train loss 0.0309158712478 | accuracy 0.991066666667
Test loss 0.0662971557525 | accuracy 0.9796
epoch 3
Train loss 0.0277806539985 | accuracy 0.992133333333
Test loss 0.0707003810546 | accuracy 0.9787
epoch 4
Train loss 0.0253880935822 | accuracy 0.993116666667
Test loss 0.0683887335024 | accuracy 0.9784
epoch 5
Train loss 0.0234229001391 | accuracy 0.99355
Test loss 0.0681526364241 | accuracy 0.9794
epoch 6
Train loss 0.021178430815 | accuracy 0.994366666667
Test loss 0.064279399443 | accuracy 0.9799
epoch 7
Train loss 0.0187197637801 | accuracy 0.995433333333
Test loss 0.0639767580499 | accuracy 0.9813
epoch 8
Train loss 0.0174248694595 | accuracy 0.995466666667
Test loss 0.0632141828359 | accuracy 0.9812
epoch 9
Train loss 0.0156684828868 | accuracy 0.99625
Test loss 0.0662767

# Part 2: MNISTをCNNで識別してみよう

## CNNとは
- 畳み込み層とプーリング層
 - 畳み込み層が色々な特徴を抽出し、プーリング層がそれを集約する。
<img src="figure/cnn.JPG", width=800>


- 畳み込みとプーリングを繰り返したら、程よいところでベクトルに引き伸ばして、全結合層(=線形層)を繋げて識別する。
 - 全結合層は英語で書くとfully-connected layerで、よく「fc層」と呼んだりする。

## MNISTデータの読み込み
- CNNを使うときは、データXはデータ数\*カラーチャンネル数\*縦\*横の4次元テンソルの形にする。
- 混乱するとアレなのでもう一回読み込みます。

In [47]:
# Load MNIST data
dir = '/data/ishimochi0/dataset/mnist/'

X_train = np.loadtxt(dir + 'train-images.txt').astype(np.float32).reshape((-1, 1, 28, 28)) / 255.
y_train = np.loadtxt(dir + 'train-labels.txt').astype(np.int32)
N_train = len(X_train)

X_test= np.loadtxt(dir + 'test-images.txt').astype(np.float32).reshape((-1, 1, 28, 28)) / 255.
y_test = np.loadtxt(dir + 'test-labels.txt').astype(np.int32)
N_test = len(X_test)

## CNNクラス
- 畳み込み層はL.Convolution2D(入力チャンネル数、出力チャンネル数、フィルタサイズ)で定義する。
- その他の引数:
 - stride: デフォルトは1。基本的に弄らなくてよい。
 - pad: デフォルトは0。初回の演習でやったように、出力サイズは入力サイズよりも小さくなるので、あまりDeeeepにはできない。それが嫌なら、padを指定しよう。 


- プーリングはF.max_pooling_2d(x, size)のように呼び出す。

In [48]:
class CNN(Chain):
    def __init__(self):
        super(CNN, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(1, 20, 5)
            self.conv2 = L.Convolution2D(20, 50, 5)
            self.fc3 = L.Linear(800, 500)
            self.fc4 = L.Linear(500, 10)
    
    def __call__(self, x, t):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(h, 2)
        h = F.relu(self.conv2(h))
        h = F.max_pooling_2d(h, 2)
    
        h = F.relu(self.fc3(h))
        self.y = self.fc4(h)
        self.loss = F.softmax_cross_entropy(self.y, t)
        
        return self.loss

## モデルの定義とoptimizerの設定
- 今回は最適化手法にMomemtumSGDを使う。
 - 現在の勾配に、過去の勾配をある割合(0.9が一般的)で加えたものを使ってパラメータを更新。
 - SGDに比べて、振動が抑えられ、安定して学習が進むようになる。
- Weight Decayも導入してみる。
 - 重みの値が大きくなることに対して罰則を加える -> 過学習を防げる

In [49]:
model = CNN() 
model.to_gpu()
optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))

## CNNの学習
- さっきと全く同じです！

In [50]:
n_epoch = 10
batchsize = 100

for epoch in range(n_epoch):
    print('epoch', epoch)
    
    # Training
    sum_loss = 0
    pred_y = []
    perm = np.random.permutation(N_train)
    
    for i in range(0, N_train, batchsize):
        x = Variable(cuda.to_gpu(X_train[perm[i: i+batchsize]]))
        t = Variable(cuda.to_gpu(y_train[perm[i: i+batchsize]]))
        
        optimizer.update(model, x, t)
        sum_loss += cuda.to_cpu(model.loss.data) * len(x.data)
        pred_y.extend(np.argmax(cuda.to_cpu(model.y.data), axis=1)) 
    
    loss = sum_loss / N_train
    accuracy = np.sum(np.eye(10)[pred_y] * np.eye(10)[y_train[perm]]) / N_train
    print('Train loss', loss, '| accuracy', accuracy)  
    
    
    # Testing
    sum_loss = 0
    pred_y = []
    
    for i in range(0, N_test, batchsize):
        x = Variable(cuda.to_gpu(X_test[i: i+batchsize]))
        t = Variable(cuda.to_gpu(y_test[i: i+batchsize]))
        
        sum_loss += cuda.to_cpu(model(x, t).data) * len(x.data)
        pred_y.extend(np.argmax(cuda.to_cpu(model.y.data), axis=1))

    loss = sum_loss / N_test
    accuracy = np.sum(np.eye(10)[pred_y] * np.eye(10)[y_test]) / N_test
    print('Test loss ', loss, '| accuracy ', accuracy) 

epoch 0
Train loss 0.210450467332 | accuracy 0.937416666667
Test loss  0.0608491955325 | accuracy  0.9804
epoch 1
Train loss 0.0624794593376 | accuracy 0.98025
Test loss  0.0453843436053 | accuracy  0.9856
epoch 2
Train loss 0.0454200434165 | accuracy 0.985666666667
Test loss  0.0498993334919 | accuracy  0.9841
epoch 3
Train loss 0.035279705843 | accuracy 0.989016666667
Test loss  0.0318839562109 | accuracy  0.9901
epoch 4
Train loss 0.0289386086546 | accuracy 0.9913
Test loss  0.035740500268 | accuracy  0.9883
epoch 5
Train loss 0.0250872532746 | accuracy 0.992283333333
Test loss  0.0328098152308 | accuracy  0.9893
epoch 6
Train loss 0.0227422051321 | accuracy 0.993066666667
Test loss  0.0351577963938 | accuracy  0.9885
epoch 7
Train loss 0.0192432374331 | accuracy 0.994233333333
Test loss  0.0301677836797 | accuracy  0.9886
epoch 8
Train loss 0.0177193273854 | accuracy 0.99495
Test loss  0.0293604841243 | accuracy  0.9897
epoch 9
Train loss 0.0162710440721 | accuracy 0.995016666667
T

# Part 3: CIFAR-10をCNNで識別してみよう

## CIFAR-10の読み込み
- 飛行機、自動車、鳥、猫、鹿、犬、蛙、馬、船、トラックの10クラス
- 訓練データ50000枚、テストデータ10000枚
- 32\*32のRGB画像
- 研究室のサーバから読み込む

<img src="figure/cifar-10.JPG", width=500>

In [55]:
import pickle
import sys

def unpickle(file):
    with open(file, 'rb') as fp:
        if sys.version_info.major == 2:
            data = pickle.load(fp)
        elif sys.version_info.major == 3:
            data = pickle.load(fp, encoding='latin-1')
    return data

train = [unpickle('/data/ishimochi0/dataset/cifar-10-batches-py/data_batch_%d' %i) for i in range(1,6)]
X_train_ = np.concatenate([d['data'] for d in train]).reshape((-1, 3, 32, 32)).astype('float32') / 255.
y_train = np.concatenate([d['labels'] for d in train]).astype('int32')
N_train = len(X_train_)

test = unpickle('/data/ishimochi0/dataset/cifar-10-batches-py/test_batch')
X_test_ = test['data'].reshape((-1, 3, 32, 32)).astype('float32') / 255.
y_test = np.array(test['labels'], dtype='int32')
N_test = len(X_test_)

## データの前処理
255で割るだけでなく、学習データの平均を引くと2〜3%くらい精度が上がります！
- データの分布が0中心になる->ReLUの非線形性をフル活用できる
- 各カラーチャンネルごとに平均を計算

In [54]:
def preprocess(X_train_, X_test_):
    X_mean = np.mean(X_train_, axis=(0, 2, 3), keepdims=True)
    X_train = X_train_ - X_mean
    X_test = X_test_ - X_mean
    return X_train, X_test

## CNNクラス
- 自分でネットワークを構築してみよう。

In [56]:
class CNN(Chain):
    def __init__(self):
        super(CNN, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 32, 5, pad=2)
            self.conv2 = L.Convolution2D(32, 64, 5, pad=2)
            self.conv3 = L.Convolution2D(64, 128, 5, pad=2)
            self.fc4 = L.Linear(16*128, 128)
            self.fc5 = L.Linear(128, 10)
    
    def __call__(self, x, t):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(h, 2)
        h = F.relu(self.conv2(h))
        h = F.max_pooling_2d(h, 2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(h, 2)
    
        h = F.relu(self.fc4(h))
        self.y = self.fc5(h)
        self.loss = F.softmax_cross_entropy(self.y, t)
        
        return self.loss

## モデルの定義とoptimizerの設定
- さっきと同じです！

In [57]:
model = CNN() 
model.to_gpu()
optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))

## CIFAR-10の学習
- データの前処理が加わりました！

In [58]:
X_train, X_test = preprocess(X_train_, X_test_)

n_epoch = 20
batchsize = 100

for epoch in range(n_epoch):
    print('epoch', epoch)
    
    # Training
    sum_loss = 0
    pred_y = []
    perm = np.random.permutation(N_train)
    
    for i in range(0, N_train, batchsize):
        x = Variable(cuda.to_gpu(X_train[perm[i: i+batchsize]]))
        t = Variable(cuda.to_gpu(y_train[perm[i: i+batchsize]]))
        
        optimizer.update(model, x, t)
        sum_loss += cuda.to_cpu(model.loss.data) * len(x.data)
        pred_y.extend(np.argmax(cuda.to_cpu(model.y.data), axis=1)) 
    
    loss = sum_loss / N_train
    accuracy = np.sum(np.eye(10)[pred_y] * np.eye(10)[y_train[perm]]) / N_train
    print('Train loss', loss, '| accuracy', accuracy)  
    
    
    # Testing
    sum_loss = 0
    pred_y = []
    
    for i in range(0, N_test, batchsize):
        x = Variable(cuda.to_gpu(X_test[i: i+batchsize]))
        t = Variable(cuda.to_gpu(y_test[i: i+batchsize]))
        
        sum_loss += cuda.to_cpu(model(x, t).data) * len(x.data)
        pred_y.extend(np.argmax(cuda.to_cpu(model.y.data), axis=1))

    loss = sum_loss / N_test
    accuracy = np.sum(np.eye(10)[pred_y] * np.eye(10)[y_test]) / N_test
    print('Test loss ', loss, '| accuracy ', accuracy)

epoch 0
Train loss 1.58841254425 | accuracy 0.42392
Test loss  1.27095008016 | accuracy  0.5339
epoch 1
Train loss 1.15325602698 | accuracy 0.58908
Test loss  1.04236194789 | accuracy  0.6243
epoch 2
Train loss 0.940291112542 | accuracy 0.66822
Test loss  0.900817964673 | accuracy  0.681
epoch 3
Train loss 0.786138306975 | accuracy 0.72376
Test loss  0.87589012742 | accuracy  0.6987
epoch 4
Train loss 0.678492396355 | accuracy 0.76382
Test loss  0.811496055722 | accuracy  0.7245
epoch 5
Train loss 0.586351692796 | accuracy 0.79342
Test loss  0.775932582617 | accuracy  0.7366
epoch 6
Train loss 0.505986744761 | accuracy 0.82432
Test loss  0.742292573452 | accuracy  0.751
epoch 7
Train loss 0.427829537928 | accuracy 0.8503
Test loss  0.790488277674 | accuracy  0.7493
epoch 8
Train loss 0.361187461138 | accuracy 0.87356
Test loss  0.829762749374 | accuracy  0.7341
epoch 9
Train loss 0.298232761681 | accuracy 0.895
Test loss  0.834604891539 | accuracy  0.744
epoch 10
Train loss 0.240132400

# Part 4: CIFAR-10で高精度を目指そう

- 普通に実装すると、CIFAR-10の識別精度は75%程度
- これでも昔に比べたら十分凄いが、いろいろ工夫すると、80%、90%と精度を伸ばすことができる

### 1. ネットワークを深くする
先ほどの解答では畳み込み層が3層、全結合層が2層だったが、これをもっと増やす。
- 畳み込み層の増やし方

畳み込み層では、padを入れることでサイズを維持できることはすでに学んだ。  
しかし、プーリング層では、必ずサイズが半分になる。  
CIFAR-10の画像サイズは32\*32なので、プーリングはせいぜい3回が限界。これは変えられない。  
では、畳み込み層の数も3層が限界？？


畳み込み層とプーリング層が交互に来る必要はない。  
プーリング層とプーリング層の間には、畳み込み層が何層あっても良い！   


そのときに、サイズの小さい畳み込み層を複数積み重ねるとよい。
たとえば、

```python
conv1 = L.Convolution2D(3, 32, 5, pad=2),
```

としていたところを、
```python
conv11 = L.Convolution2D(3, 32, 3, pad=1),
conv12 = L.Convolution2D(32, 32, 3, pad=1),
```
のようにする。  
フィルタサイズ5x5の畳み込みを1回するのと、3x3を2回するのでは、やっていることはほとんど同じだが、  
間にReLUが入ることで非線形性が増し、表現力が上がると言われている。

あとは、畳み込み層のチャンネル数を増やしてみるとどうなるか？？  
いろいろ試してみてください。

- 全結合層の増やし方

3層にするのが一般的。2層目は入力次元数と出力次元数を同じにすることが多い。

### 2. Dropoutを入れる
Dropoutとは、学習時にランダムに選ばれた半分(または任意の割合)のニューロンを無視して学習を行うテクニック。  
毎回違うニューロンが学習に回されるので、学習データに過剰に適合してしまう「過学習」を防ぐことができる。  
テスト時は全ニューロンを用いるので、各ニューロンの出力を半分にしてやる必要がある。  
基本的に全結合層(最終層以外)に使用する。

- Chainerでの書き方

call関数内で、F.dropoutで使用できる。学習時とテスト時で挙動が違うので、学習時かどうかを引数に与える必要がある。  
無視する割合はデフォルトで0.5なので指定しなくて良い。
```python
def __call__(self, x, t, train):
    # 略
    h = F.relu(self.fc4(h))
    h = F.dropout(h, train=train)  
    # 略
```

### 3. Batch Normalizationを入れる
Batch Normalization (BN)は、各層の出力をチャンネルごとに正規化(=平均0、分散1にすること)するテクニック。  
層を多層にすると、データの値の分布が途中でどんどん変なところへ行ってしまい、学習が上手くいかない。  
BNを入れると、それを防げる。

<img src="figure/bn.JPG", width=350>


学習時はミニバッチの平均と分散を計算して正規化する。  
テスト時は、学習時に溜めておいた平均と分散を用いて正規化する。  


実際には、正規化するだけではなく、さらにパラメータ$\gamma$と$\beta$を用いてデータを拡大・平行移動している。  
この$\gamma$と$\beta$が学習すべきパラメータ。チャンネル数分に対応した分だけの$\gamma$と$\beta$がある。

基本的に畳み込み層の後に全部入れるとよい。全結合層には入れない。

- Chainerでの書き方

init関数内で、L.BatchNormalization(チャンネル数)で定義できる。
```python
super(CNN, self).__init__(
            conv1 = L.Convolution2D(3, 32, 5, pad=2),
            norm1 = L.BatchNormalization2D(32),
            # 略
        )
```

Dropoutと同様に、学習時とテスト時で挙動が違うので、call関数内で呼び出す際には引数が必要。BNでは、テスト時かどうかを表す引数が必要。  
順番がややこしいが、conv -> norm -> reluの順。

```python
def __call__(self, x, t, train):
    h = F.relu(self.norm(self.conv1(x), test=not train))
    # 略
```

### 4. Learning rateを調整する
learning rate = 1回の学習で重みをどれだけ更新するか

- 初期値

小さすぎると最初のどうでもいい穴にハマってしまうので、最初は大きめのlearning rateで学習をしたほうがよい。  
広い世界を冒険した方が、より低いところを見つけられる可能性が高まる。  
ただし、あまりにも大きすぎると発散してしまうので注意。  
いろいろ試してみてもいいが、CIFAR-10に対してはおそらく0.01がベスト。

<img src="figure/lr1.png", width=500>

- 学習の終盤

ずっと初期値のままで学習を進めていくと、今度は逆に最終的に入りたい穴に入ることができない。  
そこで、学習が佳境に差し掛かったら、学習係数を10分の1とかにしてやる。  
上手くいくと、その瞬間に精度がカクンと上がります！

<img src="figure/lr2.png", width=400>

### 5. Data augmentation
学習データを擬似的に水増しする手法。  
もしかしたら一番識別精度に効いてくる部分かもしれないので、是非いろいろ試してほしいです。


よく使われるのが、画像の一部を切り取って入力するcropping、画像のスケールを変えて入力するscaling、左右反転して入力するflippingの3つ。  
他にも、回転させたり、色を微妙に変えたりすることもある。

あらかじめaugmentした画像を用意するのでは画像の枚数が増えすぎてしまうので、  
ミニバッチごとに切り取る位置やスケールなどのパラメータを生成してaugmentするのが普通。  

また、学習時だけでなく、テスト時にも同じようにいろんなパラメータでaugmentして、全部の画像を入力し、  
得られた最終層の値の平均を取って識別するということをすると、精度がさらに上がる。